Extracting 3000 tweets from Twitter using API credentials. Hashtags used: #Corona #BJP #Congress

In [0]:
!pip install preprocessor

  Created wheel for preprocessor: filename=preprocessor-1.1.3-cp36-none-any.whl size=4478 sha256=568a06befde48613cf93888fc83a5d4fde4ce39ca5b4c27111499813c8d37338
  Stored in directory: /root/.cache/pip/wheels/98/c1/a2/21fbcfd80d76576bbf148991a66f00730f541f265c7600000f
Successfully built preprocessor


In [0]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p

In [0]:

consumer_key = 'b8R1IqFWumzNa3xcyHbo4zFxD'
consumer_secret = '5GzK94Lowdnc4W9o8MW8yCdpqUnsc53VfFufck7D7lYNoZq7G7'
access_key = '1032017296729038854-uXUp9KF8VEq5ZWYe8LFQkwKT8c336E'
access_secret = 'dUHTXKUHro2vdiHsteQyHwSZfegyhpjlxRjsxQ83NvlCb'

In [0]:
import tweepy

In [0]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [0]:
#declare file paths as follows for three files
corona_tweets = "data/corona_data_extraction/corona_data.csv"
bjp_tweets = "data/bjp_data_extraction/bjp_data.csv"
congress_tweets = "data/congress_data_extraction/congress_data.csv"

In [0]:
#columns of the csv file
COLS = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
'favorite_count', 'retweet_count', 'original_author',   'possibly_sensitive', 'hashtags',
'user_mentions', 'place', 'place_coord_boundaries']

In [0]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [0]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [0]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [0]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [0]:
import preprocessor as p

In [0]:
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [0]:
def write_tweets(keyword, file):
    #If the file exists, then read the existing data from the CSV file.
    if os.path.exists(file):
        df = pd.read_csv(file, header=0)
    else:
        df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.search, q=keyword,
                              count=200, include_rts=False):
        for status in page:
          new_entry = []
          status = status._json
        if status['lang'] != 'en':
           continue
        if status[‘created_at’] in df[‘created_at’].values:
           i = df.loc[df[‘created_at’] == status[‘created_at’]].index[0]
        if status[‘favorite_count’] != df.at[i, ‘favorite_count’] or \
            status[‘retweet_count’] != df.at[i, ‘retweet_count’]:
            df.at[i, ‘favorite_count’] = status[‘favorite_count’]
            df.at[i, ‘retweet_count’] = status[‘retweet_count’]
            continue